<a href="https://colab.research.google.com/github/Victorious3/vanila-gan/blob/master/Generative_Adversarial_Networks_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Example

In [ ]:
!pip install tensorboardX
from utils import Logger

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
import torchvision
from torchvision import transforms
from torchvision.transforms import  ToTensor, Normalize, Compose
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import imageio
import numpy
from utils import Logger
...

Ellipsis

## Dataset
To simplify, the PyTorch MNIST wrapper, which downloads and loads the MNIST dataset. See the [documentation](https://github.com/pytorch/vision/blob/master/torchvision/datasets/mnist.py) for more information about the interface. The default parameters will take 5,000 of the training examples and place them into a validation dataset. Test Change The data will be saved into a folder called `MNIST_data`. 

In [ ]:

MNIST_data = torchvision.datasets.MNIST(
    root='./data/MNIST_data',
    train=True,
    download=True,
    transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=(0.5), std=(0.5) ) ] )
)

MNIST_loader = torch.utils.data.DataLoader(
    MNIST_data, batch_size=128, shuffle=True
)

img, label = MNIST_data[0]
print('Label: ', label)
print(img[:,10:15,10:15])
torch.min(img), torch.max(img)



Label:  5
tensor([[[-0.9922,  0.2078,  0.9843, -0.2941, -1.0000],
         [-1.0000,  0.0902,  0.9843,  0.4902, -0.9843],
         [-1.0000, -0.9137,  0.4902,  0.9843, -0.4510],
         [-1.0000, -1.0000, -0.7255,  0.8902,  0.7647],
         [-1.0000, -1.0000, -1.0000, -0.3647,  0.8824]]])


(tensor(-1.), tensor(1.))

## Random Noise
Generate uniform noise from -1 to 1 with shape `[batch_size, dim]`. Implement `sample_noise` Hint: use `torch.rand`. Make sure noise is the correct shape and type:

In [ ]:

def sample_noise(batch_size, dim): 
  n = Variable(torch.rand(batch_size, dim) + torch.rand(batch_size, dim)*(-1))
  if torch.cuda.is_available(): 
    return n.cuda() 
  return n
 
sample_noise(5000,128)

tensor([[-0.5695,  0.0816,  0.6032,  ..., -0.1096, -0.0971,  0.1423],
        [ 0.3310, -0.0955,  0.1415,  ..., -0.0018,  0.6924, -0.3429],
        [ 0.0922, -0.2482,  0.2171,  ...,  0.1988, -0.1546,  0.4738],
        ...,
        [-0.1146, -0.0061,  0.2677,  ...,  0.1389,  0.2187, -0.7277],
        [-0.6628,  0.0865,  0.2640,  ...,  0.1631, -0.0979, -0.4196],
        [ 0.4532, -0.1037,  0.1297,  ..., -0.7978,  0.1063, -0.6563]])

# Discriminator
Our first step is to build a discriminator. Fill in the architecture: A three hidden-layer discriminative neural network.
 * Fully connected layer 
 * LeakyReLU 
 * Fully connected layer 
 * LeakyReLU 
 * Fully connected layer 

In [ ]:
class DiscriminatorNet(torch.nn.Module):

    def __init__(self):
        super(DiscriminatorNet, self).__init__()
        in_feat = 784
        out_feat = 1

        self.hidden0 = nn.Sequential(
            nn.Linear(in_feat, 1024),
            nn.LeakyReLU(0.2)
        )
        
        self.hidden1 = nn.Sequential(
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2)
        )

        self.hidden2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2)
        )

        self.out = nn.Sequential(
            torch.nn.Linear(256, out_feat),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x
    discriminator = DiscriminatorNet()

# Generator
Similar like above:
 * Fully connected layer
 * `ReLU`
 * Fully connected layer
 * `ReLU`
 * Fully connected layer
 * `TanH` (to clip the image to be in the range of [-1,1])
 

In [ ]:
class GeneratorNet(torch.nn.Module):

    def __init__(self):
        super(GeneratorNet, self).__init__()
        in_feat = 256
        out_feat = 784

        self.hidden0 = nn.Sequential(
            nn.Linear(in_feat, 256),#do y=Ax+b
            nn.ReLU()
        )

        self.hidden1 = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU()
        )

        self.hidden2 = nn.Sequential(
            nn.Linear(512, 1024),
            nn.ReLU()
        )

        self.out = nn.Sequential(
            nn.Linear(1024, out_feat),
            nn.Tanh()
        )        


    def forward(self, x):
        x = self.hidden0(x)
        x = self.hidden1(x)
        x = self.hidden2(x)
        x = self.out(x)
        return x

    generator = GeneratorNet()

# Optimization
Make a function that returns an `optim.Adam` optimizer

In [ ]:
# Optimizers
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

# Loss function
loss = nn.BCELoss()

# Number of steps to apply to the discriminator
d_steps = 1
# Number of epochs
num_epochs = 10


# Training a GAN!



In [ ]:
def real_data_target(size):

    data = Variable(torch.ones(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

def fake_data_target(size):

    data = Variable(torch.zeros(size, 1))
    if torch.cuda.is_available(): return data.cuda()
    return data

In [ ]:
def train_discriminator(optimizer, real_data, fake_data):
    # Reset gradients
    optimizer.zero_grad()

    # Train on Real Data
    real_prediction = discriminator(real_data)

    # Calculate error and backpropagate
    real_error = loss(real_prediction, real_data_target(real_data.size(0)))
    real_error.backward()

    # Train on Fake Data
    fake_prediction = discriminator(fake_data)

    # Calculate error and backpropagate 
    fake_error = loss(fake_prediction, fake_data_target(real_data.size(0)))
    fake_error.backward()

    # Update weights with gradients
    optimizer.step()
    
    # Return error
    return real_error + fake_error, real_prediction, fake_prediction
    
def train_generator(optimizer, fake_data):
    # Reset gradients
    optimizer.zero_grad()
    # Sample noise and generate fake data
    prediction = discriminator(fake_data)
    # Calculate error and backpropagate
    size = prediction.size(0)
    error = loss(prediction, real_data_target(size))
    error.backward()
    # Update weights with gradients
    optimizer.step()
    # Return error
    return error

In [ ]:
logger = Logger(model_name='VGAN', data_name='MNIST')
num_batches = 12
num_test_samples = 8

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28)

for epoch in range(num_epochs):
    for n_batch, (real_batch,_) in enumerate(MNIST_loader):

        # Train Discriminator
        real_data = Variable(real_batch.view(real_batch.size(0), 784))
        # if torch.cuda.is_available(): real_data = real_data.cuda()
        # Generate fake data
        fake_data = generator(sample_noise( real_data.size(0), num_batches)).detach()
        # Train D
        d_error, d_pred_real, d_pred_fake = train_discriminator(d_optimizer, real_data, fake_data)

        # Train Generator
        # Generate fake data
        fake_data = generator(sample_noise(real_batch.size(0), num_batches))
        # Train G
        g_error = train_generator(g_optimizer, fake_data)
        # Log error
        logger.log(d_error, g_error, epoch, n_batch, num_batches)
        
        if (n_batch) % 300 == 0: 
            test_images = vectors_to_images(generator(sample_noise(256,256)))
            test_images = test_images.data
            logger.log_images(test_images, num_test_samples,epoch, n_batch, num_batches);
            # Display status Logs
            logger.display_status(epoch, num_epochs, n_batch, num_batches,d_error, g_error, d_pred_real, d_pred_fake)

        # Model Checkpoints
        logger.save_models(generator, discriminator, epoch)

RuntimeError: ignored